# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 20 2022 10:54AM,242315,21,402502733-1,"NBTY Global, Inc.",Released
1,Jul 20 2022 10:50AM,242313,15,VT80175416,"Virtus Pharmaceuticals, LLC",Released
2,Jul 20 2022 10:50AM,242313,15,VT80175417,"Virtus Pharmaceuticals, LLC",Released
3,Jul 20 2022 10:50AM,242312,15,PNC377393A,"Person & Covey, Inc.",Released
4,Jul 20 2022 10:47AM,242311,15,8295338,"Carwin Pharmaceutical Associates, LLC",Released
5,Jul 20 2022 10:47AM,242311,15,8295339,"Carwin Pharmaceutical Associates, LLC",Released
6,Jul 20 2022 10:47AM,242311,15,8295340,"Carwin Pharmaceutical Associates, LLC",Released
7,Jul 20 2022 10:47AM,242311,15,8299144,"Carwin Pharmaceutical Associates, LLC",Released
8,Jul 20 2022 10:47AM,242311,15,8299145,"Carwin Pharmaceutical Associates, LLC",Released
9,Jul 20 2022 10:47AM,242311,15,8299146,"Carwin Pharmaceutical Associates, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,242311,Released,6
31,242312,Released,1
32,242313,Released,2
33,242314,Released,1
34,242315,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242311,NaN,NaN,6.0
242312,NaN,NaN,1.0
242313,NaN,NaN,2.0
242314,NaN,NaN,1.0
242315,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242207,0.0,1.0,0.0
242224,0.0,0.0,1.0
242229,9.0,15.0,1.0
242231,0.0,0.0,1.0
242233,0.0,0.0,19.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242207,0,1,0
242224,0,0,1
242229,9,15,1
242231,0,0,1
242233,0,0,19


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242207,0,1,0
1,242224,0,0,1
2,242229,9,15,1
3,242231,0,0,1
4,242233,0,0,19


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242207,,1,
1,242224,,,1
2,242229,9,15,1
3,242231,,,1
4,242233,,,19


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 20 2022 10:54AM,242315,21,"NBTY Global, Inc."
1,Jul 20 2022 10:50AM,242313,15,"Virtus Pharmaceuticals, LLC"
3,Jul 20 2022 10:50AM,242312,15,"Person & Covey, Inc."
4,Jul 20 2022 10:47AM,242311,15,"Carwin Pharmaceutical Associates, LLC"
10,Jul 20 2022 10:46AM,242314,18,Innova Softgel LLC
11,Jul 20 2022 10:44AM,242310,15,"Brookfield Pharmaceuticals, LLC"
36,Jul 20 2022 10:43AM,242309,15,"Alliance Pharma, Inc."
52,Jul 20 2022 10:40AM,242308,10,"Senseonics, Incorporated"
53,Jul 20 2022 10:35AM,242307,19,"AdvaGen Pharma, Ltd"
54,Jul 20 2022 10:28AM,242306,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 20 2022 10:54AM,242315,21,"NBTY Global, Inc.",,,1
1,Jul 20 2022 10:50AM,242313,15,"Virtus Pharmaceuticals, LLC",,,2
2,Jul 20 2022 10:50AM,242312,15,"Person & Covey, Inc.",,,1
3,Jul 20 2022 10:47AM,242311,15,"Carwin Pharmaceutical Associates, LLC",,,6
4,Jul 20 2022 10:46AM,242314,18,Innova Softgel LLC,,,1
5,Jul 20 2022 10:44AM,242310,15,"Brookfield Pharmaceuticals, LLC",,,25
6,Jul 20 2022 10:43AM,242309,15,"Alliance Pharma, Inc.",,,16
7,Jul 20 2022 10:40AM,242308,10,"Senseonics, Incorporated",,,1
8,Jul 20 2022 10:35AM,242307,19,"AdvaGen Pharma, Ltd",,1,
9,Jul 20 2022 10:28AM,242306,10,"Citieffe, Inc.",,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 20 2022 10:54AM,242315,21,"NBTY Global, Inc.",1,,
1,Jul 20 2022 10:50AM,242313,15,"Virtus Pharmaceuticals, LLC",2,,
2,Jul 20 2022 10:50AM,242312,15,"Person & Covey, Inc.",1,,
3,Jul 20 2022 10:47AM,242311,15,"Carwin Pharmaceutical Associates, LLC",6,,
4,Jul 20 2022 10:46AM,242314,18,Innova Softgel LLC,1,,
5,Jul 20 2022 10:44AM,242310,15,"Brookfield Pharmaceuticals, LLC",25,,
6,Jul 20 2022 10:43AM,242309,15,"Alliance Pharma, Inc.",16,,
7,Jul 20 2022 10:40AM,242308,10,"Senseonics, Incorporated",1,,
8,Jul 20 2022 10:35AM,242307,19,"AdvaGen Pharma, Ltd",,1,
9,Jul 20 2022 10:28AM,242306,10,"Citieffe, Inc.",2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 20 2022 10:54AM,242315,21,"NBTY Global, Inc.",1,,
1,Jul 20 2022 10:50AM,242313,15,"Virtus Pharmaceuticals, LLC",2,,
2,Jul 20 2022 10:50AM,242312,15,"Person & Covey, Inc.",1,,
3,Jul 20 2022 10:47AM,242311,15,"Carwin Pharmaceutical Associates, LLC",6,,
4,Jul 20 2022 10:46AM,242314,18,Innova Softgel LLC,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 20 2022 10:54AM,242315,21,"NBTY Global, Inc.",1.0,NaN,NaN
1,Jul 20 2022 10:50AM,242313,15,"Virtus Pharmaceuticals, LLC",2.0,NaN,NaN
2,Jul 20 2022 10:50AM,242312,15,"Person & Covey, Inc.",1.0,NaN,NaN
3,Jul 20 2022 10:47AM,242311,15,"Carwin Pharmaceutical Associates, LLC",6.0,NaN,NaN
4,Jul 20 2022 10:46AM,242314,18,Innova Softgel LLC,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 20 2022 10:54AM,242315,21,"NBTY Global, Inc.",1.0,0.0,0.0
1,Jul 20 2022 10:50AM,242313,15,"Virtus Pharmaceuticals, LLC",2.0,0.0,0.0
2,Jul 20 2022 10:50AM,242312,15,"Person & Covey, Inc.",1.0,0.0,0.0
3,Jul 20 2022 10:47AM,242311,15,"Carwin Pharmaceutical Associates, LLC",6.0,0.0,0.0
4,Jul 20 2022 10:46AM,242314,18,Innova Softgel LLC,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2665078,115.0,1.0,0.0
15,1211555,50.0,0.0,0.0
16,969164,4.0,0.0,0.0
18,484538,2.0,0.0,0.0
19,1695913,6.0,16.0,9.0
20,484535,32.0,0.0,0.0
21,242315,1.0,0.0,0.0
22,242275,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2665078,115.0,1.0,0.0
1,15,1211555,50.0,0.0,0.0
2,16,969164,4.0,0.0,0.0
3,18,484538,2.0,0.0,0.0
4,19,1695913,6.0,16.0,9.0
5,20,484535,32.0,0.0,0.0
6,21,242315,1.0,0.0,0.0
7,22,242275,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,115.0,1.0,0.0
1,15,50.0,0.0,0.0
2,16,4.0,0.0,0.0
3,18,2.0,0.0,0.0
4,19,6.0,16.0,9.0
5,20,32.0,0.0,0.0
6,21,1.0,0.0,0.0
7,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,115.0
1,15,Released,50.0
2,16,Released,4.0
3,18,Released,2.0
4,19,Released,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21,22
Status,,,,,,,,
Completed,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0
Executing,1.0,0.0,0.0,0.0,16.0,0.0,0.0,1.0
Released,115.0,50.0,4.0,2.0,6.0,32.0,1.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0
1,Executing,1.0,0.0,0.0,0.0,16.0,0.0,0.0,1.0
2,Released,115.0,50.0,4.0,2.0,6.0,32.0,1.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0
1,Executing,1.0,0.0,0.0,0.0,16.0,0.0,0.0,1.0
2,Released,115.0,50.0,4.0,2.0,6.0,32.0,1.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()